In [128]:
!pip install pandas
!pip install optuna
!pip install optuna-dashboard
!pip install scikit
!pip install imbalanced-learn
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    PowerTransformer,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
    minmax_scale,
)
from sklearn.metrics import recall_score, accuracy_score,f1_score, precision_score, roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.ensemble import IsolationForest
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
import warnings
import optuna
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score

ERROR: Could not find a version that satisfies the requirement scikit (from versions: none)
ERROR: No matching distribution found for scikit


In [129]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using", device)
def init_weights(model): #tested already
    if isinstance(model, nn.Linear):  # Apply only to linear layers
        nn.init.xavier_uniform_(model.weight)
        if model.bias is not None:
            nn.init.zeros_(model.bias)
            
def fold_to_dataloader_tensor(train_x, test_x, train_y, test_y, batch_size=64, device=device):
    train_dataset = TensorDataset(
        torch.tensor(train_x.values,dtype=torch.float32).to(device), 
        torch.tensor(train_y.values,dtype=torch.float32).to(device))
    val_dataset = TensorDataset(
        torch.tensor(test_x.values,dtype=torch.float32).to(device), 
        torch.tensor(test_y.values,dtype=torch.float32).to(device))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False, drop_last=True)
    return train_loader, val_loader 

def get_feature_count(loader):
    """returns the number of features in the dataset"""
    return next(iter(loader))[0].shape[1]

from Criterion_Models import *
def criterion_mapping(criterion_choice:str, pos_weight:float=None, alpha:float=None, gamma:float=None):
    """
    Feel free to add any custom loss functions here.
    returns function for criterion
    """
    if criterion_choice == "FocalLoss":
        return FocalLoss(alpha =alpha, gamma=gamma)
    elif criterion_choice == "DiceLoss":
        return DiceLoss()
    elif criterion_choice == "BCEWithLogitsLoss":
        return nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight])) if pos_weight else nn.BCEWithLogitsLoss()
    return nn.BCEWithLogitsLoss() 

Using cpu


In [130]:
random_state = 42
raw_dataset = pd.read_csv("./data/processed_data.csv") #data has X and Y
X = raw_dataset.drop(columns=["DR"])
Y = pd.DataFrame(raw_dataset["DR"])

In [131]:
#drop cols here
# numeric_columns = ['Age', 'UAlb', 'Ucr', 'UACR', 'TC', 'TG', 'TCTG', 'LDLC', 'HDLC', 'Scr', 'BUN', 'FPG', 'HbA1c', 'Height', 'Weight', 'BMI', 'Duration']
# binary_columns = ['Gender', 'DR', 'Community_baihe', 'Community_chonggu', 'Community_huaxin', 'Community_jinze', 'Community_liantang', 'Community_xianghuaqiao', 'Community_xujin', 'Community_yingpu', 'Community_zhaoxian', 'Community_zhujiajiao']

In [132]:
X_FOR_FOLDS, X_FINAL_TEST, Y_FOR_FOLDS, Y_FINAL_TEST = train_test_split(X, Y, test_size=0.1, random_state=random_state, stratify=Y)

In [133]:
def augment_data_in_place(X, X_test, Y=None, normalisation_method=MinMaxScaler(), robust_scaler=RobustScaler(), noise=None):
    """
    Apply preprocessing transformations to data in this order:
    1. Optional log transformation for numerical features
    2. RobustScaler to handle outliers
    3. MinMaxScaler to normalize to [0,1] range
    4. Optional noise addition to negative class samples
    
    Parameters:
    -----------
    X : DataFrame
        Training features
    X_test : DataFrame
        Test features
    Y : DataFrame or Series, optional
        Target variable (needed if adding noise selectively)
    normalisation_method : sklearn scaler, default=MinMaxScaler()
        Final normalization method
    robust_scaler : sklearn scaler, default=RobustScaler()
        Initial scaling method to handle outliers
    noise : float, optional
        Standard deviation of Gaussian noise to add to negative class samples
        
    Returns:
    --------
    X_transformed, X_test_transformed : DataFrames
        Transformed training and test data
    """
    # Create copies to avoid modifying original data
    X_transformed = X.copy()
    X_test_transformed = X_test.copy()
    
    # Identify numerical columns (exclude binary ones)
    all_numerical_columns = ['Age', 'UAlb', 'Ucr', 'UACR', 'TC', 'TG', 'TCTG', 'LDLC', 
                           'HDLC', 'Scr', 'BUN', 'FPG', 'HbA1c', 'Height', 'Weight', 
                           'BMI', 'Duration']
    binary_columns = ['Gender', 'DR', 'Community_baihe', 'Community_chonggu', 
                    'Community_huaxin', 'Community_jinze', 'Community_liantang', 
                    'Community_xianghuaqiao', 'Community_xujin', 'Community_yingpu', 
                    'Community_zhaoxian', 'Community_zhujiajiao']
    
    # Find which numerical columns actually exist in the data
    numeric_cols = [col for col in all_numerical_columns if col in X.columns and col in X_test.columns]
    
    if not numeric_cols:
        print("No matching numerical columns found for transformation.")
        # Apply standard normalization to all data if no specific columns matched
        X_normalized = pd.DataFrame(normalisation_method.fit_transform(X), columns=X.columns, index=X.index)
        X_test_normalized = pd.DataFrame(normalisation_method.transform(X_test), columns=X_test.columns, index=X_test.index)
        return X_normalized, X_test_normalized
    
    # 1. Apply log transformation to numeric columns (handling zeros/negatives with log1p)
    # Skip log transform for any columns that have negative values
    for col in numeric_cols:
        if (X_transformed[col].min() >= 0) and (X_test_transformed[col].min() >= 0):
            X_transformed[col] = np.log1p(X_transformed[col])
            X_test_transformed[col] = np.log1p(X_test_transformed[col])
    
    # 2. Apply RobustScaler to handle outliers (only to numeric columns)
    if robust_scaler:
        robust_scaled_train = robust_scaler.fit_transform(X_transformed[numeric_cols])
        robust_scaled_test = robust_scaler.transform(X_test_transformed[numeric_cols])
        
        X_transformed[numeric_cols] = robust_scaled_train
        X_test_transformed[numeric_cols] = robust_scaled_test
    
    # 3. Apply final normalization method (to all columns)
    X_normalized = normalisation_method.fit_transform(X_transformed)
    X_test_normalized = normalisation_method.transform(X_test_transformed)
    
    # Convert back to DataFrame with original column names
    X_normalized = pd.DataFrame(X_normalized, columns=X.columns, index=X.index)
    X_test_normalized = pd.DataFrame(X_test_normalized, columns=X_test.columns, index=X_test.index)
    
    # 4. Add noise ONLY to negatives (class 0) if Y is provided and noise is set
    if noise is not None and noise > 0 and Y is not None:
        # Identify negative class indices (class 0)
        if isinstance(Y, pd.DataFrame):
            negative_indices = Y[Y.iloc[:, 0] == 0].index
        else:  # Handle Series case
            negative_indices = Y[Y == 0].index
            
        # Add noise only to negative samples and only to numerical columns
        if len(negative_indices) > 0:
            noise_matrix = np.random.normal(0, noise, (len(negative_indices), len(numeric_cols)))
            X_normalized.loc[negative_indices, numeric_cols] += noise_matrix
            
            # Ensure values stay within [0,1] range after noise addition
            X_normalized.loc[negative_indices, numeric_cols] = np.clip(
                X_normalized.loc[negative_indices, numeric_cols], 0, 1)
    
    return X_normalized, X_test_normalized

In [134]:
def iso_forest(X_train, Y_train, contamination=None, random_state=42):
    # Create copies of input data
    X_train_cleaned, Y_train_cleaned = X_train.copy(), Y_train.copy()
    
    # Handle Y_train whether it's a DataFrame or Series
    if isinstance(Y_train, pd.DataFrame):
        class_values = Y_train.iloc[:, 0]
    else:  # It's a Series
        class_values = Y_train
    
    # Split data by class
    X_train_zeros = X_train[class_values == 0]
    X_train_ones = X_train[class_values == 1]
    Y_train_zeros = Y_train[class_values == 0]
    Y_train_ones = Y_train[class_values == 1]
    
    # Print debug info
    print(f"Class distribution - Zeros: {len(X_train_zeros)}, Ones: {len(X_train_ones)}")
    
    # Check if we have enough samples to apply Isolation Forest
    if len(X_train_zeros) <= 1:
        print("Not enough majority class samples for Isolation Forest. Skipping.")
        return X_train, Y_train
    
    # Apply Isolation Forest to majority class only
    if X_train_zeros.isna().any().any():
        print("Warning: NaN values in the training set")
    
    # Apply Isolation Forest to majority class only
    iso_forest = IsolationForest(contamination=contamination, random_state=random_state)
    try:
        outliers = iso_forest.fit_predict(X_train_zeros)
    except Exception as e:
        print(f"Error during IsolationForest fitting: {e}")
        return X_train, Y_train  # Return original data if it fails
    
    # Keep only non-outlier majority samples
    X_train_zeros = X_train_zeros[outliers == 1]
    Y_train_zeros = Y_train_zeros[outliers == 1]
    
    print(f"After Isolation Forest - Kept {len(X_train_zeros)} out of {len(outliers)} majority samples")
    
    # Combine the cleaned majority class with the untouched minority class
    X_train_cleaned = pd.concat([X_train_zeros, X_train_ones])
    Y_train_cleaned = pd.concat([Y_train_zeros, Y_train_ones])
    
    return X_train_cleaned, Y_train_cleaned

def FOLDS_GENERATOR(X, Y, normalisation_method=MinMaxScaler(), n_splits=5, random_state=None, oversampler=None, contamination=0.05, noise=None):
    """
    Generates stratified folds with specified normalization.
    """
    kF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    kFolds_list = []

    robust_scaler = RobustScaler()
    
    # Convert column names to strings to ensure compatibility
    X = X.copy()
    X.columns = [str(col) for col in X.columns]

    for fold, (train_idx, test_idx) in enumerate(kF.split(X, Y)):
        # Split the data into training and testing sets for this fold
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        Y_train, Y_test = Y.iloc[train_idx], Y.iloc[test_idx]
        
        # Initialize cleaned data with original data
        X_train_cleaned, Y_train_cleaned = X_train.copy(), Y_train.copy()
        
        # Apply isolation forest if contamination is specified
        if contamination is not None and contamination > 0:
            X_train_cleaned, Y_train_cleaned = iso_forest(X_train, Y_train, contamination=contamination, random_state=random_state)
        
        # Data augmentation on cleaned data
        X_train_scaled, X_test_scaled = augment_data_in_place(X_train_cleaned, X_test, Y_train_cleaned, 
                                                             normalisation_method=normalisation_method, 
                                                             robust_scaler=robust_scaler, 
                                                             noise=noise)
        
        # Handle oversampling if needed
        print("Before oversampling class distribution:")
        print(Y_train_cleaned.value_counts())
        if oversampler:
            X_train_scaled, Y_train_cleaned = oversampler.fit_resample(X_train_scaled, Y_train_cleaned)
        print("\nAfter oversampling class distribution:")
        print(Y_train_cleaned.value_counts())
        

        # Handle community columns with safeguards
        # First check if columns are string type
        community_cols = []
        for col in X_train_scaled.columns:
            col_str = str(col)  # Convert to string if not already
            if isinstance(col_str, str) and col_str.startswith('Community'):
                community_cols.append(col)

        if community_cols:
            try:
                X_train_scaled[community_cols] = X_train_scaled[community_cols].apply(
                    lambda row: pd.Series(np.eye(len(row))[row.argmax()], index=community_cols), axis=1
                )
            except Exception as e:
                print(f"Warning: Error processing community columns: {e}. Skipping one-hot encoding.")

        # Ensure 'Gender' is still binary (0 or 1) if it exists
        if 'Gender' in X_train_scaled.columns:
            X_train_scaled['Gender'] = (X_train_scaled['Gender'] > 0.5).astype(int)
            X_test_scaled['Gender'] = (X_test_scaled['Gender'] > 0.5).astype(int)

        # Append the processed fold to the list
        kFolds_list.append((X_train_scaled, X_test_scaled, Y_train_cleaned, Y_test))

        print(f"Fold: {fold+1}, Train: {X_train_scaled.shape}, Test: {X_test_scaled.shape}")

    return kFolds_list

In [135]:
scaling_pipeline = Pipeline([
    ('robust', RobustScaler()),
    ('minmax', MinMaxScaler())
])

oversampler = ADASYN(sampling_strategy='minority', n_neighbors=5, random_state=42)
contamination = 0.05

# 1. Sanity checks
# print("Features Data Shape:", X_FOR_FOLDS.shape)
# print("Labels Data Shape:", Y_FOR_FOLDS.shape)
if X_FOR_FOLDS.shape[0] == 0 or Y_FOR_FOLDS.shape[0] == 0:
    raise ValueError("Input data is empty. Please check your data preprocessing.")

# 2. Convert NumPy arrays to pandas DataFrames if needed
if not isinstance(X_FOR_FOLDS, pd.DataFrame):
    # print("Converting X_FOR_FOLDS from NumPy array to DataFrame")
    column_names = ['age', 'image1', 'DR1', 'CSME1', 'HBPRP1', 'image2', 'DR2', 'CSME2', 'HBPRP2', 'WorseDR', 'DR', 'UACR', 'TCTG', 'TC', 'LDLC', 'TG', 'HDLC', 'Scr', 'Ucr', 'BUN', 'UAlb', 'FPG', 'HbA1c', 'Height', 'Weight', 'BMI', 'Duration', 'Gender_2', 'Community_chonggu', 'Community_huaxin', 'Community_jinze', 'Community_liantang', 'Community_xianghuaqiao', 'Community_xujin', 'Community_yingpu', 'Community_zhaoxian', 'Community_zhujiajiao']
    # Create with basic column names if needed
    X_FOR_FOLDS = pd.DataFrame(X_FOR_FOLDS, columns=column_names)

    X_FOR_FOLDS.rename(columns={'age': 'Age',
                           'ualb': 'UAlb',
                           'ucr': 'Ucr',
                           'uacr': 'UACR',
                           'tc': 'TC',
                           'tg': 'TG',
                           'tctg': 'TCTG',
                           'ldlc': 'LDLC',
                           'hdlc': 'HDLC',
                           'scr': 'Scr',
                           'bun': 'BUN',
                           'fpg': 'FPG',
                           'hba1c': 'HbA1c',
                           'height': 'Height',
                           'weight': 'Weight',
                           'bmi': 'BMI',
                           'duration': 'Duration',
                           'Gender_2': 'Gender'}, inplace=True)

if not isinstance(Y_FOR_FOLDS, (pd.DataFrame, pd.Series)):
    # print("Converting Y_FOR_FOLDS from NumPy array to Series")
    Y_FOR_FOLDS = pd.Series(Y_FOR_FOLDS)

# # 3. Inspect NaNs
# # print("NaNs in features:\n", X_FOR_FOLDS.isna().sum())
# # print("NaNs in labels:\n", Y_FOR_FOLDS.isna().sum())

# # 4. Find non-numeric columns - but first check data types
# # print("Data types in X_FOR_FOLDS:")
# # print(X_FOR_FOLDS.dtypes)

# Skip string processing if no object columns exist
string_cols = X_FOR_FOLDS.select_dtypes(include=['object']).columns
if len(string_cols) > 0:
    # print(f"Found {len(string_cols)} object columns: {list(string_cols)}")
    for col in string_cols:
        # Only apply string operations if the column contains strings
        if pd.api.types.is_string_dtype(X_FOR_FOLDS[col]):
            X_FOR_FOLDS[col] = X_FOR_FOLDS[col].str.replace(',', '', regex=False)
        X_FOR_FOLDS[col] = pd.to_numeric(X_FOR_FOLDS[col], errors='coerce')
else:
    # print("No string columns found - skipping string processing")
    pass

# 5. Check if any columns are still non-numeric
still_obj = X_FOR_FOLDS.select_dtypes(include=['object']).columns
if len(still_obj) > 0:
    # print(f"Dropping non‑numeric columns: {list(still_obj)}")
    X_FOR_FOLDS = X_FOR_FOLDS.drop(columns=still_obj)  # Use assignment instead of inplace

# 6. Fill NaNs (from original missing or failed coercion)
X_FOR_FOLDS = X_FOR_FOLDS.fillna(0)  # Use assignment instead of inplace

# 7. Now call your fold generator
# print("Ready to generate folds. X shape:", X_FOR_FOLDS.shape, "Y shape:", Y_FOR_FOLDS.shape)
kFolds = FOLDS_GENERATOR(
    X_FOR_FOLDS,
    Y_FOR_FOLDS,
    normalisation_method=MinMaxScaler(),
    n_splits=5,
    oversampler=oversampler,
    random_state=42,
    contamination=contamination,
    noise=None
)
# print(f"\nNumber of folds generated: {len(kFolds)}")

for fold_idx, (X_train_scaled, X_test_scaled, Y_train_cleaned, Y_test) in enumerate(kFolds):
    # print(f"\n--- Fold {fold_idx + 1} ---")

    # print("\nColumn names in X_train_scaled:", X_train_scaled.columns.tolist())
    # print("Column names in X_test_scaled:", X_test_scaled.columns.tolist())

    # Check feature scaling (0 to 1) for training data
    print("\nTraining Data Feature Scaling - Fold:", fold_idx + 1)
    print("Min values:\n", X_train_scaled.min())
    print("Max values:\n", X_train_scaled.max())

    # Check feature scaling (0 to 1) for testing data
    print("\nTesting Data Feature Scaling - Fold:", fold_idx + 1)
    print("Min values:\n", X_test_scaled.min())
    print("Max values:\n", X_test_scaled.max())

    # Check if 'Gender' column is binary in training data
    if 'Gender' in X_train_scaled.columns:
        print("\n'Gender' column in Training Data - Fold:", fold_idx + 1)
        print(X_train_scaled['Gender'].unique())

    # Check if 'Gender' column is binary in testing data
    if 'Gender' in X_test_scaled.columns:
        print("\n'Gender' column in Testing Data - Fold:", fold_idx + 1)
        print(X_test_scaled['Gender'].unique())

    # Check if 'Community' columns are binary in training data
    community_cols_train = [col for col in X_train_scaled.columns if col.startswith('Community')]
    if community_cols_train:
        print("\n'Community' columns in Training Data - Fold:", fold_idx + 1)
        for col in community_cols_train:
            print(f"{col}: {X_train_scaled[col].unique()}")

    # Check if 'Community' columns are binary in testing data
    community_cols_test = [col for col in X_test_scaled.columns if col.startswith('Community')]
    if community_cols_test:
        print("\n'Community' columns in Testing Data - Fold:", fold_idx + 1)
        for col in community_cols_test:
            print(f"{col}: {X_test_scaled[col].unique()}")

Class distribution - Zeros: 4129, Ones: 464
After Isolation Forest - Kept 3922 out of 4129 majority samples
Before oversampling class distribution:
DR 
0.0    3922
1.0     464
Name: count, dtype: int64

After oversampling class distribution:
DR 
1.0    3979
0.0    3922
Name: count, dtype: int64
Fold: 1, Train: (7901, 28), Test: (1149, 28)
Class distribution - Zeros: 4129, Ones: 464
After Isolation Forest - Kept 3922 out of 4129 majority samples
Before oversampling class distribution:
DR 
0.0    3922
1.0     464
Name: count, dtype: int64

After oversampling class distribution:
DR 
1.0    3960
0.0    3922
Name: count, dtype: int64
Fold: 2, Train: (7882, 28), Test: (1149, 28)
Class distribution - Zeros: 4130, Ones: 464
After Isolation Forest - Kept 3923 out of 4130 majority samples
Before oversampling class distribution:
DR 
0.0    3923
1.0     464
Name: count, dtype: int64

After oversampling class distribution:
DR 
1.0    3963
0.0    3923
Name: count, dtype: int64
Fold: 3, Train: (7886,

In [136]:
for list in kFolds:
    print(list[0].shape, list[1].shape, list[2].shape, list[3].shape)
    print(list[0].describe())
    a = list[0]
    break

(7901, 28) (1149, 28) (7901, 1) (1149, 1)
               Age       Gender         UAlb          Ucr         UACR  \
count  7901.000000  7901.000000  7901.000000  7901.000000  7901.000000   
mean      0.594660     0.537780     0.389796     0.465419     0.356018   
std       0.122816     0.498602     0.191754     0.373218     0.165705   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.524505     0.000000     0.260280     0.134245     0.245180   
50%       0.609557     1.000000     0.361727     0.200581     0.330989   
75%       0.676143     1.000000     0.501705     0.882666     0.445693   
max       1.000000     1.000000     1.000000     1.000000     1.000000   

                TC           TG         TCTG         LDLC         HDLC  ...  \
count  7901.000000  7901.000000  7901.000000  7901.000000  7901.000000  ...   
mean      0.435022     0.185039     0.434814     0.562047     0.424684  ...   
std       0.097940     0.082591     0.155653     0.119

In [137]:
oversampler = ADASYN(sampling_strategy='minority', n_neighbors=5, random_state=42)
contamination = 0.05
normalisation_method = MinMaxScaler()
kFolds2 = FOLDS_GENERATOR(X_FOR_FOLDS, Y_FOR_FOLDS, 
                         normalisation_method = normalisation_method, 
                         n_splits=5, 
                         oversampler = oversampler, random_state=42, contamination=contamination, noise = 0.2)

Class distribution - Zeros: 4129, Ones: 464
After Isolation Forest - Kept 3922 out of 4129 majority samples
Before oversampling class distribution:
DR 
0.0    3922
1.0     464
Name: count, dtype: int64

After oversampling class distribution:
DR 
1.0    4001
0.0    3922
Name: count, dtype: int64
Fold: 1, Train: (7923, 28), Test: (1149, 28)
Class distribution - Zeros: 4129, Ones: 464
After Isolation Forest - Kept 3922 out of 4129 majority samples
Before oversampling class distribution:
DR 
0.0    3922
1.0     464
Name: count, dtype: int64

After oversampling class distribution:
DR 
1.0    3973
0.0    3922
Name: count, dtype: int64
Fold: 2, Train: (7895, 28), Test: (1149, 28)
Class distribution - Zeros: 4130, Ones: 464
After Isolation Forest - Kept 3923 out of 4130 majority samples
Before oversampling class distribution:
DR 
0.0    3923
1.0     464
Name: count, dtype: int64

After oversampling class distribution:
DR 
0.0    3923
1.0    3894
Name: count, dtype: int64
Fold: 3, Train: (7817,

In [138]:
for list in kFolds2:
    print(list[0].shape, list[1].shape, list[2].shape, list[3].shape)
    print(list[0].describe())
    break

(7923, 28) (1149, 28) (7923, 1) (1149, 1)
               Age       Gender         UAlb          Ucr         UACR  \
count  7923.000000  7923.000000  7923.000000  7923.000000  7923.000000   
mean      0.596533     0.545122     0.413025     0.493828     0.377046   
std       0.182528     0.497991     0.235354     0.373827     0.217835   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.492103     0.000000     0.244837     0.134245     0.226818   
50%       0.606012     1.000000     0.392682     0.408707     0.361061   
75%       0.705448     1.000000     0.571080     0.883003     0.515149   
max       1.000000     1.000000     1.000000     1.000000     1.000000   

                TC           TG         TCTG         LDLC         HDLC  ...  \
count  7923.000000  7923.000000  7923.000000  7923.000000  7923.000000  ...   
mean      0.437548     0.201240     0.432193     0.558524     0.424255  ...   
std       0.171568     0.145735     0.205710     0.184